In [2]:
from  selenium import webdriver 
from bs4 import BeautifulSoup as bs
import re 
from translate import Translator
import string
import re
from time import  sleep
from selenium.common.exceptions import TimeoutException
import pickle
from collections import deque
from selenium.webdriver.common.by import By
import pandas as pd
import sys
sys.setrecursionlimit(10**6)
import datetime

In [3]:
def driver():
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome('D:\coding-area\olx_scrapping\chromedriver.exe',chrome_options=chrome_options)
    return driver

In [4]:
def getfunc(link):
    while True:
        try:
            driver.get(link)
            x=driver.page_source
            return x
        except:
            print("time_out")
            sleep(60)

In [5]:
def ads_list(page_Source):
    try:
        funcSoup=bs(page_Source,'html5lib')
        xc=funcSoup.find_all('article',class_="_7e3920c1")
        list_of_ads=['https://www.olx.com.eg/'+i.find('a')['href'] for i in xc]
        return list_of_ads
    except  Exception as e:
        print( str(e),"ad_list")

In [6]:
def next_page(page_source):
    try:
        Soup=bs(page_source,'html5lib')
        x=Soup.find_all('a',class_="d73e7494")[-1]
        if  x.find('div')['title']=='Next':
            return "https://www.olx.com.eg"+x['href']
        else:
            print('the end')
    except Exception as e :
        print(str(e)+' next')

In [7]:
def extract_data(page_source,i):
    res={}
    amenities={'Balcony': 0,
 'Built in Kitchen Appliances': 0,
 'Private Garden': 0,
 'Central A/C & heating': 0,
 'Security': 0,
 'Covered Parking': 0,
 'Maids Room': 0,
 'Pets Allowed': 0,
 'Pool': 0,
 'Electricity Meter': 0,
 'Water Meter': 0,
 'Natural Gas': 0,
 'Landline': 0,
 'Elevator': 0}
    
    AddSoup=bs(page_source,'html5lib')
    try:
        location=AddSoup.find(class_="_8918c0a8")
        res.update({'location':location.text})        
    except:
        res.update({'location':location})
    try:    
        title=AddSoup.find(class_="a38b8112")
        res.update({'title':title.text})
    except:
        res.update({'title':title})
    try:
        x=AddSoup.find(attrs={'aria-label':"Creation date"}).text
        current = datetime.datetime.now()
        if 'hours' in x or 'hour' in x:
            day_diff=datetime.datetime.now()
        if 'day'  in x or 'days'  in x:
            day_diff=current-datetime.timedelta(days=int(x[0]))
        if 'week'in x or 'weeks' in x:
            day_diff=current-datetime.timedelta(weeks=int(x[0]))
        if 'month'in x or 'months' in x:
            day_diff=current-datetime.timedelta(months=int(x[0]))
        res.update({'date':day_diff})
    except Exception as e :
        print('date error:',str(e))
        res.update({'date':x})
    try: 
        all=AddSoup.findAll(class_='_59317dec')
    except:
        res.update({'page_source':page_source})
    try:
        details={i.findAll('span')[0].text:i.findAll('span')[1].text for i in all[0].find_all(class_="b44ca0b3")}
        res.update(details)
    except:
        res.update({'details':all[0]})
    if len(all)==2:
        try:
            Description=all[1].find('div' ,class_="_0f86855a")
            res.update({'Description':Description.text})
        except:
            res.update({'Description':all[1]})
            
    elif  len(all)==3:
        try:
            # src=all[1].find('div' ,class_="_0f86855a")
            amenities_found={i.text.strip():1 for i in all[2].find('div').find_all('span')}
            amenities.update(amenities_found)
            res.update(amenities)
        except:
            print(all[2])
            res.update(all[2])
        try:
            Description=all[1].find('div' ,class_="_0f86855a").text
            res.update({'Description':Description.text})
        except:
            res.update({'Description':all[1]})
        res.update({'link':i})
        return res
def dollar_extractor():
    page_source=getfunc('https://www.cbe.org.eg/en/EconomicResearch/Statistics/Pages/ExchangeRatesListing.aspx')
    soup=bs(page_source,'html5lib')
    return soup.find(class_="odd").find_all('td')[1:]
    

In [8]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome('chromedriver.exe',chrome_options=chrome_options)
link="https://www.olx.com.eg/en/ad/%D8%B4%D9%82%D9%87-%D8%A3%D9%84%D8%AA%D8%B1%D8%A7-%D8%B3%D9%88%D8%A8%D8%B1-%D9%84%D9%88%D9%83%D8%B3-%D9%84%D9%84%D8%A8%D9%8A%D8%B9-ID177682597.html"

zx=extract_data(getfunc(link),link)

C:\Users\Moham\AppData\Local\Temp\ipykernel_6092\2347946052.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe',chrome_options=chrome_options)
C:\Users\Moham\AppData\Local\Temp\ipykernel_6092\2347946052.py:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver.exe',chrome_options=chrome_options)


In [9]:
zx

{'location': 'Kafr al-Sheikh City',
 'title': 'شقه ألترا سوبر لوكس للبيع',
 'date': datetime.datetime(2022, 4, 24, 15, 26, 26, 21441),
 'Type': 'Apartment',
 'Price': '1,600,000',
 'Price Type': 'Negotiable',
 'Bedrooms': '3',
 'Bathrooms': '2',
 'Area (m²)': '180',
 'Level': '7',
 'Furnished': 'No',
 'Payment Option': 'Cash',
 'Balcony': 1,
 'Built in Kitchen Appliances': 0,
 'Private Garden': 0,
 'Central A/C & heating': 0,
 'Security': 1,
 'Covered Parking': 0,
 'Maids Room': 0,
 'Pets Allowed': 0,
 'Pool': 0,
 'Electricity Meter': 1,
 'Water Meter': 1,
 'Natural Gas': 1,
 'Landline': 1,
 'Elevator': 1,
 'Description': <div class="_59317dec"><span class="_7978e49c _2e82a662">Description</span><div class="_0f86855a"><span>شقه تشطيب ألترا سوبر لوكس برج الشرق للتأمين الدور السابع الشقه ناصيه ٣غرف مساحه  ٥×٦ +٢حمام +مطبخ مساحه ٥×٥٠٥ الشقه نموذج مميز</span></div></div>,
 'link': 'https://www.olx.com.eg/en/ad/%D8%B4%D9%82%D9%87-%D8%A3%D9%84%D8%AA%D8%B1%D8%A7-%D8%B3%D9%88%D8%A8%D8%B1-%D9%8

In [11]:
import datetime
from dateutil.relativedelta import relativedelta
current = datetime.datetime.now()

x=current-datetime.timedelta(days=1)

datetime.datetime(2022, 4, 19, 11, 17, 8, 340035)

In [19]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome('D:\coding-area\olx_scrapping\chromedriver.exe',chrome_options=chrome_options)
link="https://www.olx.com.eg/en/properties/apartments-duplex-for-sale/el-tagamoa/?filter=type_eq_1"
try:
    with open(f"{str(today)}.pkl",'rb') as dd:
        dollar,today_date= pickle.load(dd)
except:
    dollar,today_date=dollar_extractor(),today
if today_date!=today:
    dollar,today_date=dollar_extractor(),today
try:
    with open(f"statestr{str(today)}.pkl",'rb') as file:
        all_data, stack, last_link = pickle.load(file)
except:
    all_data, stack, last_link = [], deque(), link

stack.appendleft(last_link)

while stack:
    page_number=False
    link=stack.pop()
    last_link=link
    pa=re.findall('page=[0-9]+', link)
    if len(pa)!=0:
        page_number=pa=='page=2'  
    src=getfunc(link)
    print(len(all_data),link)
    if '/ad/' in link and link not in {x['link'] for x in all_data}:
        data=extract_data(src,link)
        all_data.append(data)
        with open(f"statestr{str(today)}.pkl",'wb') as file:
            state = all_data, stack, last_link
            pickle.dump(state, file)
        sleep(3)
    else:
        link=next_page(src)
        if link and page_number:
            i=0
            stack.appendleft(links[i])
            i+=1
        if link:
            stack.appendleft(link)
        stack.extendleft(ads_list(src))

C:\Users\Moham\AppData\Local\Temp\ipykernel_26876\821770463.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('D:\coding-area\olx_scrapping\chromedriver.exe',chrome_options=chrome_options)
C:\Users\Moham\AppData\Local\Temp\ipykernel_26876\821770463.py:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('D:\coding-area\olx_scrapping\chromedriver.exe',chrome_options=chrome_options)


0 https://www.olx.com.eg/en/properties/apartments-duplex-for-sale/el-tagamoa/?filter=type_eq_1
0 https://www.olx.com.eg/en/properties/apartments-duplex-for-sale/el-tagamoa/?page=2&filter=type_eq_1
0 https://www.olx.com.eg//en/ad/%D8%A7%D8%B3%D8%AA%D9%84%D8%A7%D9%85-%D9%81%D9%88%D8%B1%D9%8A-%D9%81%D9%8A-%D8%A7%D9%84%D8%AA%D8%AC%D9%85%D8%B9-%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3-%D8%B4%D9%82%D9%87-165%D9%85-%D8%A8%D9%85%D9%82%D8%AF%D9%85-250-%D8%A7%D9%84%D9%81-%D8%A8%D8%AC%D9%88%D8%A7%D8%B1-point-90-ID188336173.html
1 https://www.olx.com.eg//en/ad/%D8%A7%D8%B3%D8%AA%D8%AB%D9%85%D8%B1-%D9%81%D9%84%D9%88%D8%B3%D9%83-%D9%81%D9%8A-%D8%B4%D9%82%D8%A9-%D9%84%D9%82%D8%B7%D8%A9-%D9%85%D8%AA%D8%B4%D8%B7%D8%A8%D8%A9-137-%D9%85-%D9%85%D9%86-%D8%B3%D9%88%D8%AF%D9%8A%D9%83-%D9%81%D9%8A%D9%84%D9%8A%D8%AA-%D8%A7%D9%84%D8%AA%D8%AC%D9%85%D8%B9-%D9%88%D9%82%D8%B3%D8%B7-%D9%85%D8%B1%D9%8A%D8%AD-ID190200976.html


TypeError: 'NoneType' object is not subscriptable